In [4]:
import requests
import numpy as np
import pandas as pd
from datetime import datetime
import time
from tqdm import tqdm
import pickle

In [5]:
login = 'e.khmelevskoy@gmail.com'
token = 'a933d970677550750996dc9a4cf5a81b'
city = ['city=Москва', 'city1=Москва']
category = ['category_id=1', 'param[1957]=Вторичка']
type_adv = ['nedvigimost_type=1']
main = [f'https://ads-api.ru/main/api?user={login}&token={token}']


LIST_KEYS = [('Cсылка', 'url'), 
             ('Цена', 'price'), 
             ('Время загрузки', 'time'), 
             ('Ближайшее метро', 'metro'), 
             ('Адрес', 'address'), 
             ('Этаж', 'param_2113'), 
             ('Площадь кухни', 'param_12721'), 
             ('Расстояние до метро', 'km_do_metro'), 
             ('Количество комнат', 'param_1945'),
             ('Описание', 'description'), 
             ('Жилая площадь', 'param_12722'), 
             ('Вид объекта', 'param_1957'), 
             ('Тип дома', 'param_2009'), 
             ('Площадь', 'param_2313'), 
             ('Адрес', 'param_2314'),
             ('Город', 'city1'),
             ('Регион', 'region'),
             ('ID', 'avitoid'),
             ('ID', 'source_id')
             ]

def get_flat(json):
    flat = {}
    for name, key in LIST_KEYS:
        if key in json:
            if name in flat and flat[name] != json[key]:
                flat[name] = [flat[name], json[key]]
            elif name not in flat:
                flat[name] = json[key]
    
    for Key in ('params', 'params2', 'coords'):
        
        if Key in json:
            for key, item in json[Key].items():
                if key not in flat:
                    flat[key] = item
    return flat

def get_response(date_1, date_2):
    dates = [f'date1={date_1}', f'date2={date_2}']
    url = '&'.join([*main, *city, *dates, *category, *type_adv])
    try:
        response = requests.get(url)
        dict_flats = response.json()
        if dict_flats['code'] == 200: 
            if len(dict_flats['data']) > 0:
                return [get_flat(json) for json in dict_flats['data']]
            else:
                print('Данных нет :(')
        else:
            print(f'Ошибка {code}')
    except:
        pass

In [6]:
START_DATE = '2021-03-01'
END_DATE = datetime.today()

In [29]:
dates = pd.date_range(START_DATE, END_DATE, freq = '12h').tolist()

for i in tqdm(range(len(dates)-1)):
    d1, d2 = dates[i], dates[i+1]
    response = get_response(d1, d2)
    
    if response is not None:
        for flat in response:
            with open('flats_data.pickle', 'ab') as f:
                pickle.dump(flat, f)
    
    time.sleep(5.1)

100%|██████████████████████████████████████████████████████████████████████████████| 376/376 [1:01:37<00:00,  9.83s/it]
